# Grouping Daily Data by County and State

This will allow us to export `json` files that can be used in the website to display choropleth maps.

In [1]:
from datetime import datetime
import json as json

import numpy as np
import pandas as pd

## Loading Data

### Video Data

In [2]:
with open('usa_videos.json', 'r') as f:
    data = json.load(f)

### County Data

In [3]:
url_county_data = 'data/us_county_data.csv'

df_counties = pd.read_csv(url_county_data)
df_counties = df_counties[['FIPS_Code', 'State', 'Area_name']]
df_counties = df_counties.rename(columns={'FIPS_Code': 'fips',
                                          'State': 'state_code',
                                          'Area_name': 'county'})
df_counties.head()

,fips,state_code,county
0,0,US,United States
1,1000,AL,Alabama
2,1001,AL,Autauga County
3,1003,AL,Baldwin County
4,1005,AL,Barbour County


In [4]:
df_counties[df_counties['state_code'] == 'DC'].head()

,fips,state_code,county
331,11000,DC,District of Columbia
332,11001,DC,District of Columbia


In [5]:
df_state_abrevs = df_counties[df_counties['fips'] % 1000 == 0].reset_index()
df_state_abrevs = df_state_abrevs.rename(columns={'county': 'state'})
df_state_abrevs = df_state_abrevs[['state_code', 'state']]
df_state_abrevs.head(51)

,state_code,state
0,US,United States
1,AL,Alabama
2,AK,Alaska
3,AZ,Arizona
4,AR,Arkansas
5,CA,California
6,CO,Colorado
7,CT,Connecticut
8,DE,Delaware
9,DC,District of Columbia


In [6]:
df_counties = df_counties.merge(df_state_abrevs, on='state_code')
# Remove State FIPS
df_counties = df_counties[df_counties['fips'] % 1000 != 0]
df_counties.head()

,fips,state_code,county,state
2,1001,AL,Autauga County,Alabama
3,1003,AL,Baldwin County,Alabama
4,1005,AL,Barbour County,Alabama
5,1007,AL,Bibb County,Alabama
6,1009,AL,Blount County,Alabama


## Cleaning Data

### Creating a DataFrame

In [7]:
geo_list = [{
    'timestamp': datetime.fromisoformat(p['timestamp']),
    'lat': p['location']['lat'],
    'lon': p['location']['lon'],
    'state': p['location']['admin1'],
    'county': p['location']['admin2'],
} for p in data['data_points']]

df = pd.DataFrame(geo_list)
print(len(df))
df.head()

59026


,timestamp,lat,lon,state,county
0,2011-08-01 22:22:40,39.47353,-118.77737,Nevada,Churchill County
1,2011-11-11 21:36:54,39.31762,-74.5946,New Jersey,Atlantic County
2,2012-12-24 22:50:15,33.57786,-101.85517,Texas,Lubbock County
3,2012-12-30 22:55:39,42.7262,-71.19089,Massachusetts,Essex County
4,2013-02-09 18:23:05,42.95947,-85.48975,Michigan,Kent County


### Cleaning County Names

County names don't match perfectly between the two databases. The following counties are issues:
* All states:
    * `'... city'` -> `'City of ...'`
    * `'St. ... County'` -> `'Saint ... County'`
    * `'Ste. ... County'` -> `'Sainte ... County'`
* Alaska:
    * `'Prince of Wales-Outer Ketchikan Census A'` -> `'Annette Island Reserve'`
* Louisiana:
    * `'... Parish'` -> `'... County'`
* Louisiana, Texas, Illinois:
    * `'La Salle County'` -> `'LaSalle County'`
* Mississippi, Florida:
    * `'DeSoto County'` -> `'De Soto County'`
* New York:
    * `'Bronx County'` -> `'Bronx'`
    * `'New York County'` -> `''`
* District of Columbia:
    * State is `'Washington, D.C.'` in the dataframe
    * `'District of Columbia'` -> `''`

In [8]:
def correct_city_of(s):
    if s.startswith('City of'):
        return s[8:] + ' city'
    return s


def clean_county_names(data):
    df = data.copy()
    
    # Clean missing counties for D.C.
    df = df.replace('Washington, D.C.', 'District of Columbia')
    df.loc[(df['state'] == 'District of Columbia'), 'county'] = 'District of Columbia'
    
    # Clean missing counties for New York
    ny_missing_counties = ((df['state'] == 'New York') & (df['county'] == ''))
    df.loc[ny_missing_counties, 'county'] = 'New York County'
    
    # Parishes
    parishes = (df['state'] == 'Louisiana')
    df.loc[parishes, 'county'] = df.loc[parishes, 'county'].replace(regex=r'County$', value='Parish')
    
    # Individual replaces
    df = df.replace('Annette Island Reserve', value='Prince of Wales-Outer Ketchikan Census A')
    df = df.replace('LaSalle County', value='La Salle County')
    df = df.replace('De Soto County', value='DeSoto County')
    df = df.replace('Bronx', value='Bronx County')
    df = df.replace('City of Saint Louis', value='Saint Louis County')
    
    # All states
    df = df.replace(regex=r'^Saint ', value='St. ')
    df = df.replace(regex=r'^Sainte ', value='Ste. ')
    df.loc[:, 'county'] = df.loc[:, 'county'].apply(correct_city_of)
    return df

In [9]:
df_clean = clean_county_names(df)

In [10]:
df_clean.head()

,timestamp,lat,lon,state,county
0,2011-08-01 22:22:40,39.47353,-118.77737,Nevada,Churchill County
1,2011-11-11 21:36:54,39.31762,-74.5946,New Jersey,Atlantic County
2,2012-12-24 22:50:15,33.57786,-101.85517,Texas,Lubbock County
3,2012-12-30 22:55:39,42.7262,-71.19089,Massachusetts,Essex County
4,2013-02-09 18:23:05,42.95947,-85.48975,Michigan,Kent County


### Join DataFrames

In [11]:
data = df_clean.merge(df_counties, how='left', on=['state', 'county'])
data.head()

,timestamp,lat,lon,state,county,fips,state_code
0,2011-08-01 22:22:40,39.47353,-118.77737,Nevada,Churchill County,32001,NV
1,2011-11-11 21:36:54,39.31762,-74.5946,New Jersey,Atlantic County,34001,NJ
2,2012-12-24 22:50:15,33.57786,-101.85517,Texas,Lubbock County,48303,TX
3,2012-12-30 22:55:39,42.7262,-71.19089,Massachusetts,Essex County,25009,MA
4,2013-02-09 18:23:05,42.95947,-85.48975,Michigan,Kent County,26081,MI


In [12]:
data.isnull().values.any()

False

### Extracting Year, Month, Day

In [13]:
data['year'] = data.loc[:, 'timestamp'].apply(lambda x: x.year)
data['month'] = data.loc[:, 'timestamp'].apply(lambda x: x.month)
data['day'] = data.loc[:, 'timestamp'].apply(lambda x: x.day)
data['week'] = data.loc[:, 'timestamp'].apply(lambda x: x.week)

data = data[['timestamp', 'year', 'month', 'day', 'week',
             'lat', 'lon', 'state_code', 'state', 'county', 'fips']]

data.head()

,timestamp,year,month,day,week,lat,lon,state_code,state,county,fips
0,2011-08-01 22:22:40,2011,8,1,31,39.47353,-118.77737,NV,Nevada,Churchill County,32001
1,2011-11-11 21:36:54,2011,11,11,45,39.31762,-74.5946,NJ,New Jersey,Atlantic County,34001
2,2012-12-24 22:50:15,2012,12,24,52,33.57786,-101.85517,TX,Texas,Lubbock County,48303
3,2012-12-30 22:55:39,2012,12,30,52,42.7262,-71.19089,MA,Massachusetts,Essex County,25009
4,2013-02-09 18:23:05,2013,2,9,6,42.95947,-85.48975,MI,Michigan,Kent County,26081


## Extracting Temporal Information

### Only Keeping Data after 2020

In [14]:
data = data[data['year'] >= 2020]

In [15]:
print(len(data))

57539


### Grouping by Day and State

In [16]:
df_day_states = data.copy()
df_day_states['count'] = 1
df_day_states = df_day_states[['year', 'month', 'day', 'state_code', 'state', 'count']]
df_day_states = df_day_states.groupby(['year', 'month', 'day', 'state_code', 'state']).sum()
df_day_states = df_day_states.reset_index()

print(len(df_day_states))
df_day_states.tail(5)

8847


,year,month,day,state_code,state,count
8842,2021,1,10,UT,Utah,2
8843,2021,1,10,VA,Virginia,1
8844,2021,1,10,WA,Washington,11
8845,2021,1,10,WI,Wisconsin,1
8846,2021,1,10,WV,West Virginia,3


### Grouping by Week and State

In [17]:
df_week_states = data.copy()
df_week_states['count'] = 1
df_week_states = df_week_states[['year', 'week', 'state_code', 'state', 'count']]
df_week_states = df_week_states.groupby(['year', 'week', 'state_code', 'state']).sum()
df_week_states = df_week_states.reset_index()

print(len(df_week_states))
df_week_states.tail(5)

2160


,year,week,state_code,state,count
2155,2021,53,UT,Utah,8
2156,2021,53,VA,Virginia,8
2157,2021,53,WA,Washington,24
2158,2021,53,WI,Wisconsin,7
2159,2021,53,WV,West Virginia,2


In [18]:
df_week_states[df_week_states['state_code'] == 'TX']

,year,week,state_code,state,count
12,2020,1,TX,Texas,8
45,2020,3,TX,Texas,6
60,2020,4,TX,Texas,2
78,2020,5,TX,Texas,2
94,2020,6,TX,Texas,8
119,2020,7,TX,Texas,4
136,2020,8,TX,Texas,3
158,2020,9,TX,Texas,8
178,2020,10,TX,Texas,9
204,2020,11,TX,Texas,1


In [19]:
df_week_states[df_week_states['state_code'] == 'DC']

,year,week,state_code,state,count
85,2020,6,DC,District of Columbia,2
211,2020,12,DC,District of Columbia,2
239,2020,13,DC,District of Columbia,1
344,2020,17,DC,District of Columbia,1
379,2020,18,DC,District of Columbia,1
443,2020,20,DC,District of Columbia,1
477,2020,21,DC,District of Columbia,1
516,2020,22,DC,District of Columbia,1
607,2020,24,DC,District of Columbia,1
651,2020,25,DC,District of Columbia,1


### Grouping by Month and State

In [20]:
df_month_states = data.copy()
df_month_states['count'] = 1
df_month_states = df_month_states[['year', 'month', 'state_code', 'state', 'count']]
df_month_states = df_month_states.groupby(['year', 'month', 'state_code', 'state']).sum()
df_month_states = df_month_states.reset_index()

print(len(df_month_states))
df_month_states.tail(5)

617


,year,month,state_code,state,count
612,2021,1,VT,Vermont,1
613,2021,1,WA,Washington,110
614,2021,1,WI,Wisconsin,43
615,2021,1,WV,West Virginia,28
616,2021,1,WY,Wyoming,4


## Grouping by County

### Grouping only by County

In [25]:
df_counties = data.copy()
df_counties['count'] = 1
df_counties = df_counties[['fips', 'count']]
df_counties = df_counties.groupby(['fips']).sum()
df_counties = df_counties.reset_index()

print(len(df_counties))
df_counties.tail(5)

2340


,fips,count
2335,56033,4
2336,56035,1
2337,56037,1
2338,56039,9
2339,56041,1


In [26]:
county_counts = []
for i, row in df_counties.iterrows():
    county_info = {
        'fips': int(row.fips),
        'count': int(row['count'])
    }
    county_counts.append(county_info)

In [27]:
with open('data/videos_per_county.json', 'w') as f:
    json.dump({'video_counts': county_counts}, f)

### Grouping by County and Week

In [21]:
df_week_counties = data.copy()
df_week_counties['count'] = 1
df_week_counties = df_week_counties[['year', 'week', 'state_code', 'state', 'fips', 'county', 'count']]

# Merge 2020 and 2021 week 53
y21_w53 = ((df_week_counties['year'] == 2021) & (df_week_counties['week'] == 53))
df_week_counties.loc[y21_w53, 'year'] = 2020

# Group by year and week
df_week_counties = df_week_counties.groupby(['year', 'week', 'state_code', 'state', 'fips', 'county']).sum()
df_week_counties = df_week_counties.reset_index()

print(len(df_week_counties))
df_week_counties.tail(5)

16152


,year,week,state_code,state,fips,county,count
16147,2021,1,WV,West Virginia,54099,Wayne County,1
16148,2021,1,WY,Wyoming,56001,Albany County,1
16149,2021,1,WY,Wyoming,56005,Campbell County,1
16150,2021,1,WY,Wyoming,56017,Hot Springs County,1
16151,2021,1,WY,Wyoming,56033,Sheridan County,1


In [22]:
video_counts = {}

for year in ['2020', '2021']:
    video_counts[year] = {}
    for week in range(1, 54):
        video_counts[year][str(week)] = []

In [23]:
for i, row in df_week_counties.iterrows():
    county_info = {
        'fips': int(row.fips),
        'state': row.state,
        'county': row.county,
        'count': int(row['count'])
    }
    video_counts[str(row.year)][str(row.week)].append(county_info)

In [24]:
with open('data/videos_per_county_per_week.json', 'w') as f:
    json.dump({'video_counts': video_counts}, f)